![image](images/how.png)

#### Aquí encontrarás todo lo que necesitas saber: https://www.kaggle.com/t/997162b5cae941f08ac3d3e6ebab504e

In [1]:
!pip3 install pandas_profiling

     command: 'c:\users\maria\appdata\local\programs\python\python39\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\maria\\AppData\\Local\\Temp\\pip-install-kcaj_nq1\\llvmlite\\setup.py'"'"'; __file__='"'"'C:\\Users\\maria\\AppData\\Local\\Temp\\pip-install-kcaj_nq1\\llvmlite\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record 'C:\Users\maria\AppData\Local\Temp\pip-record-h3rh2our\install-record.txt' --single-version-externally-managed --compile --install-headers 'c:\users\maria\appdata\local\programs\python\python39\Include\llvmlite'
         cwd: C:\Users\maria\AppData\Local\Temp\pip-install-kcaj_nq1\llvmlite\
    Complete output (27 lines):
    running install
    running build
    got version from file C:\Users\maria\AppData\Local\Temp\pip-install-kcaj_nq1\llvmlite\llvmlite/_version.py {'version': '0.34.0', 

In [4]:
import pandas as pd
import numpy as np

# la otra forma, pero con mismo resultado: from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import pickle

import warnings
warnings.filterwarnings('ignore')

from sklearn.pipeline import Pipeline

from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

from sklearn.model_selection import RepeatedKFold

from sklearn.model_selection import GridSearchCV
from pandas_profiling import ProfileReport

ModuleNotFoundError: No module named 'pandas_profiling'

In [2]:
### para que funcione necesitas bajarte los archivos de datos de Kaggle :) 
df = pd.read_csv("data/diamonds_train.csv")
df

,Unnamed: 0,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.21,Ideal,H,VS2,63.0,57.0,6.73,6.70,4.23,6134
1,1,0.28,Very Good,D,VVS2,64.0,56.0,4.14,4.17,2.66,532
2,2,0.42,Premium,F,VS1,61.2,58.0,4.86,4.82,2.96,1103
3,3,0.26,Ideal,H,IF,61.1,57.0,4.16,4.12,2.53,600
4,4,1.10,Good,G,SI1,63.4,57.0,6.52,6.55,4.14,4997
...,...,...,...,...,...,...,...,...,...,...,...
40340,40340,1.55,Premium,H,VS2,61.3,61.0,7.46,7.39,4.55,11708
40341,40341,0.36,Ideal,D,SI1,60.6,56.0,4.58,4.63,2.79,619
40342,40342,0.57,Very Good,I,VS2,62.2,55.0,5.33,5.34,3.32,1267
40343,40343,1.01,Very Good,F,IF,59.6,62.0,6.47,6.56,3.88,9965


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40345 entries, 0 to 40344
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  40345 non-null  int64  
 1   carat       40345 non-null  float64
 2   cut         40345 non-null  object 
 3   color       40345 non-null  object 
 4   clarity     40345 non-null  object 
 5   depth       40345 non-null  float64
 6   table       40345 non-null  float64
 7   x           40345 non-null  float64
 8   y           40345 non-null  float64
 9   z           40345 non-null  float64
 10  price       40345 non-null  int64  
dtypes: float64(6), int64(2), object(3)
memory usage: 3.4+ MB


In [4]:
le = preprocessing.LabelEncoder()

In [5]:
cat_col = ["color", "cut", "clarity" ]
for i in cat_col:
    df[i] = le.fit_transform(df[i])


In [ ]:
profile = ProfileReport(df, title="Pandas Profiling Report")
profile

#### 1. Definir X e y

In [6]:
# Guardo los datos en las variables
X = np.array(df[["carat", "color", "cut", "clarity"]])
y = np.array(df['price'])
X.shape

(40345, 4)

In [7]:
y.shape

(40345,)

In [8]:
y

array([6134,  532, 1103, ..., 1267, 9965, 1340], dtype=int64)

#### 2. Dividir X_train, X_test, y_train, y_test

In [9]:
#Divido en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [10]:
# Create a pipeline

pipe = Pipeline(steps=[('classifier', RandomForestRegressor())])

to_test = np.arange(1, 10)
linear_params = {
    'classifier': [linear_model.LinearRegression()],
    'classifier__fit_intercept': ['False', 'True'],
    'classifier__normalize': ['False', 'True']
    }

random_n_estimators = range(0,100)
random_min_samples_split = range(0,100)
random_forest_params = {
    'classifier': [RandomForestRegressor()],
    'classifier__n_estimators': random_n_estimators,
    'classifier__max_features': [1, 2, 3],
    'classifier__min_samples_split' : random_min_samples_split
    }


# Create space of candidate learning algorithms and their hyperparameters
search_space = [
    linear_params,
    random_forest_params,
    ]

In [11]:
cv = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1)

# Create grid search 
clf = GridSearchCV(estimator=pipe, param_grid=search_space, cv=cv, verbose=0, n_jobs=-1)

# Fit grid search
best_model = clf.fit(X_train, y_train)

In [12]:
# View best model
separator = "\n############################\n"
print(separator)
print("best estimator:", best_model.best_estimator_.get_params()['classifier'])
print(separator)
print("clf.best_params_", clf.best_params_)
print(separator)


############################

best estimator: RandomForestRegressor(max_features=3, min_samples_split=12, n_estimators=53)

############################

clf.best_params_ {'classifier': RandomForestRegressor(max_features=3, min_samples_split=12, n_estimators=53), 'classifier__max_features': 3, 'classifier__min_samples_split': 12, 'classifier__n_estimators': 53}

############################



In [13]:
print("clf.best_score", clf.best_score_)

clf.best_score 0.9798344489991763


In [14]:
best_model.score(X_test, y_test) *100

97.88871008337595

In [15]:
model = RandomForestRegressor(max_features=3, min_samples_split=12, n_estimators=53)
model.fit(X, y)

RandomForestRegressor(max_features=3, min_samples_split=12, n_estimators=53)

## Una vez listo el modelo, toca predecir con el dataset de predicción 

In [16]:
to_pred = pd.read_csv("data/diamonds_test.csv")

In [17]:
# la primera columna es id_case, deberéis quitarla para hacer la predicción si la habéis quitado para entrenar
# pero guardarla para incluirla en la submission

# IMPORTANTE: APLICAR LO MISMO A ESTOS DATOS QUE HAYÁIS APLICADO A LOS DATOS DE ENTRENAMIENTO
# SI EL ARRAY CON EL QUE HICISTEIS .fit() ERA DE 4 COLUMNAS, PARA .predict() DEBEN SER LAS MISMAS
# SI AL ARRAY CON EL QUE HICISTEIS .fit() LO NORMALIZASTEIS, PARA .predict() DEBÉIS NORMALIZARLO
# TODO IGUAL SALVO BORRAR FILAS, EL NÚMERO DE ROWS SE DEBE MANTENER EN ESTE SET, PUES LA PREDICCIÓN DEBE TENER 133744 FILAS
# SI O SI

to_pred

,Unnamed: 0,carat,cut,color,clarity,depth,table,x,y,z
0,0,0.30,Ideal,H,SI2,60.0,56.0,4.41,4.43,2.65
1,1,0.34,Ideal,D,IF,62.1,57.0,4.52,4.46,2.79
2,2,1.57,Very Good,I,VS2,60.3,58.0,7.58,7.55,4.56
3,3,0.31,Ideal,H,VS2,61.8,57.0,4.32,4.36,2.68
4,4,1.51,Good,I,VVS1,64.0,60.0,7.26,7.21,4.63
...,...,...,...,...,...,...,...,...,...,...
13444,13444,1.05,Ideal,F,SI2,60.3,55.0,6.55,6.68,3.99
13445,13445,0.27,Very Good,G,VS1,61.9,56.0,4.13,4.16,2.56
13446,13446,1.50,Premium,D,VS2,61.8,60.0,7.30,7.37,4.53
13447,13447,1.70,Premium,F,SI2,60.2,61.0,7.79,7.68,4.66


In [18]:
to_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13449 entries, 0 to 13448
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  13449 non-null  int64  
 1   carat       13449 non-null  float64
 2   cut         13449 non-null  object 
 3   color       13449 non-null  object 
 4   clarity     13449 non-null  object 
 5   depth       13449 non-null  float64
 6   table       13449 non-null  float64
 7   x           13449 non-null  float64
 8   y           13449 non-null  float64
 9   z           13449 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 1.0+ MB


In [19]:
cat_col = ["color", "cut", "clarity" ]
for i in cat_col:
    to_pred[i] = le.fit_transform(to_pred[i])


In [20]:
predictions_submit = model.predict(to_pred[["carat", "color", "cut", "clarity"]])
predictions_submit

array([  417.98429572,  1740.58427227,  9464.27891935, ...,
       15079.59734686, 10185.37498999,   794.86029364])

In [23]:
# predictions_submit = le2.inverse_transform(predictions_submit)

In [24]:
# le.inverse_transform(predictions_submit)

In [22]:
submission = pd.DataFrame({"id": to_pred["Unnamed: 0"], "price": predictions_submit})
submission

,id,price
0,0,417.984296
1,1,1740.584272
2,2,9464.278919
3,3,549.236888
4,4,9045.669326
...,...,...
13444,13444,4712.445879
13445,13445,429.005872
13446,13446,15079.597347
13447,13447,10185.374990


In [23]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13449 entries, 0 to 13448
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      13449 non-null  int64  
 1   price   13449 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 210.3 KB


In [27]:
# le.inverse_transform(submission["days"])

In [24]:
sample = pd.read_csv("data/sample_submission.csv")

In [25]:
sample.shape

(13449, 2)

In [26]:
if submission.shape == sample.shape:
    if submission.columns.all() == sample.columns.all():
        if submission.id.all() == sample.id.all():
            print("you're ready to submit!")
            submission.to_csv("output/to_submit.csv", index = False)
            # ¡¡¡¡¡¡¡ADD INDEX = FALSE!!!!!!!!!
        else:
           print("error id")
    else:
        print("error columns all")
else:
    print("error shape")

you're ready to submit!
